In [60]:
import sklearn
import numpy as np
import xgboost as xgb 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [61]:
import pandas as pd
import numpy as np

In [62]:
class Node:
    def __init__(self, feature_to_split=None, threshold=None, left=None, right=None, data=None):
        self.feature_to_split = feature_to_split
        self.threshold = threshold
        self.left = left
        self.right = right
        self.data = data

In [63]:
class Custom_xgb:
    def __init__(self, n_estimators=5, learning_rate=0.1, max_depth=3, min_n_samples=2, lambda_reg=1):
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.min_n_samples = min_n_samples
        self.lambda_reg = lambda_reg
        self.trees = []
        self.base_pred = None


    def similarity(self, gradients, hessians):
        sum_grads = np.sum(gradients)
        sum_hess = np.sum(hessians)
        return (sum_grads**2)/(sum_hess + self.lambda_reg)
    

    def xgb_gain(self, g, h, g_left, h_left, g_right, h_right):
        score_root = self.similarity(g, h)
        score_left = self.similarity(g_left, h_left)
        score_right = self.similarity(g_right, h_right)
        return  0.5 * (score_left + score_right - score_root)
    

    def leaf_weight(self, gradients, hessians):
        sum_grads = np.sum(gradients)
        sum_hess = np.sum(hessians)
        return -sum_grads/(sum_hess + self.lambda_reg)
    

    def find_best_split(self, X: pd.DataFrame, gradients, hessians):        

        # store and update the bests when a better gain is found
        best_feature = None
        best_gain = -1000
        best_threshold = None
        best_mask = None

        # take a feature, then go through all values of that feature, taking them as the new threshold to check gain
        for feature in X.columns: 
            thresholds = X[feature].unique()
            for threshold in thresholds:
                # divide into y_left and y_right
                mask = (X[feature] <= threshold).values

                if sum(mask)==0 or sum(~mask)==0:
                    continue

                g_left, g_right = gradients[mask], gradients[~mask]
                h_left, h_right = hessians[mask], hessians[~mask]


                # check gain for the current threshold and feature, and update if a better gain is found
                current_gain = self.xgb_gain(gradients, hessians, g_left, h_left, g_right, h_right)

                if current_gain > best_gain:
                    best_gain = current_gain
                    best_feature = feature
                    best_threshold = threshold
                    best_mask = mask
            

        if best_feature is None:
            return None, None, None, None, None, None, None, None


        return (best_feature, best_threshold, 
                X[best_mask], gradients[best_mask], hessians[best_mask],
                X[~best_mask], gradients[~best_mask], hessians[~best_mask])
    

    def build_tree(self, X: pd.DataFrame, gradients, hessians, depth=0):
        n_samples, _ = X.shape

        # if we satisfy the conditions, we return a leaf node
        if depth >= self.max_depth or n_samples < self.min_n_samples:
            leaf_value = self.leaf_weight(gradients, hessians)
            return Node(data=leaf_value)
        
        
        
        best_feature, best_threshold, X_left, g_left, h_left, X_right, g_right, h_right = self.find_best_split(X, gradients, hessians) 

        # return a leaf node if no gain was found
        if best_feature is None:
            leaf_value = self.leaf_weight(gradients, hessians)
            return Node(data=leaf_value)


        left_child = self.build_tree(X_left, g_left, h_left, depth+1)
        right_child = self.build_tree(X_right, g_right, h_right, depth+1)

        return Node(feature_to_split=best_feature, threshold=best_threshold, left=left_child, right=right_child)


    def predict_tree(self, node, X):
        predictions = []
        for i in range(len(X)):
            row = X.iloc[i]
            current = node

            while current.data is None:
                if row[current.feature_to_split] <= current.threshold:
                    current = current.left
                else:
                    current = current.right
            predictions.append(current.data)

        return np.array(predictions)
        


    def fit(self, X, y):
        self.base_pred = np.mean(y)
        y_pred = np.full(len(y), self.base_pred)

        for _ in range(self.n_estimators):
            gradients = y_pred - y
            hessians = np.ones(len(y))

            tree = self.build_tree(X, gradients, hessians, depth=0)
            self.trees.append(tree)

            update_values = self.predict_tree(tree, X)
            y_pred += self.learning_rate*update_values




    def predict(self, X):
        final_predictions = np.full(len(X), self.base_pred)


        for tree in self.trees:
            final_predictions += self.learning_rate*self.predict_tree(tree, X)

        return final_predictions




    # def gain_variance(self, y: pd.DataFrame, y_left: pd.DataFrame, y_right: pd.DataFrame):
    #     y_var = np.var(y)
    #     y_left_var = np.var(y_left)
    #     y_right_var = np.var(y_right)

    #     weighted_var = (len(y_left)*y_left_var + len(y_right)*y_right_var)/len(y)

    #     return (y_var - weighted_var)
        

# ------------------------------------------------------------------------------------
        # gains = dict()
        # max_gain = 0

        # # for categorical data. OHE only
        # for feature in X.columns:
        #     for row in range(len(X)):
        #         if X[row][feature]:
        #             pd.concat([y_left, y[row]])
        #         else:
        #             pd.concat([y_right, y[row]])

        #     current_gain = self.gain_variance(y, y_left, y_right)

        #     if current_gain > max_gain:
        #         max_gain = current_gain
            
        #     gains[str(current_gain)] = feature

        # feature_to_split = gains[str(max_gain)]


        # X_left = pd.DataFrame()
        # X_left.columns = X.columns
        # X_right = pd.DataFrame()
        # X_right.columns = X.columns


        # for row in range(len(X)):
        #     if X[row][feature_to_split]:
        #         pd.concat([X_left, X[row]])
        #     else:
        #         pd.concat([X_right, X[row]])

        
        # for continuous (will later delete the above categorical)


        # ---------------------------------------------

    # def entropy(self, X: pd.DataFrame, feature: str):
    #     successes = np.sum(X[feature])

    #     if successes == 0 or successes == len(X):
    #         return 1
        
    #     else:
    #         p1 = np.sum(X[feature])/len(X)
    #         p2 = 1-p1
    #         return -p1*np.log2(p1) - p2*np.log2(p2)



    # def gain_entropy(self, X: pd.DataFrame, X_left: pd.DataFrame, X_right: pd.DataFrame, feature: str):
    #     x_entropy = self.entropy(X, feature)
    #     x_left_entropy = self.entropy(X_left, feature)
    #     x_right_entropy = self.entropy(X_right, feature)

    #     weighted_entropy = len(X_left)*x_left_entropy + len(X_right)*x_right_entropy

    #     return (x_entropy - weighted_entropy)
        

In [64]:
# preprocessing steps from previous notebook
df = pd.read_excel("./Curated_data_1234_datapoints.xlsx")
df.columns = df.columns.str.strip()
df = df.drop(columns=["Reference DOI", "Unnamed: 23"], axis=1)
df['Support Loading [wt.%]'] = 100 - df['Metal Loading [wt.%]'] - df['Promoter 1 loading [wt.%]'] - df['Promoter 2 loading [wt.%]']

df = df.dropna(subset=["Catalyst amount [g]"]).reset_index(drop=True)

display(df)

,STY [mgMeOH h-1 gcat-1],Family,Metal Loading [wt.%],CR Metal [pm],Support 1,MW Support 1 [g mol-1],Name of Support2,MW of Support 2 [g mol-1],Name of Support 3,MW Support 3 [g mol-1],...,Promoter 2 loading [wt.%],Calcination Temperature [K],Calcination duration [h],SBET [m2 g-1],H2/CO2 [-],GHSV [cm3 h-1 gcat-1],Catalyst amount [g],Pressure [Mpa],Temperature [K],Support Loading [wt.%]
0,2620.0,In2O3,4.4,144,Fe3O4,159.69,0,0,0,0.0,...,0.2,623,2.0,9.0,10.0,115500.0,0.50,6.0,598.0,88.6
1,2400.0,In2O3,4.4,144,Fe3O4,159.69,0,0,0,0.0,...,0.2,623,2.0,9.0,8.0,94500.0,0.50,6.0,598.0,88.6
2,2233.6,Cu,36.5,138,ZnO,81.38,0,0,SBA-15,60.0,...,0.0,573,4.0,218.0,3.0,120000.0,0.02,4.0,453.0,60.8
3,2230.0,In2O3,4.4,144,Fe3O4,159.69,0,0,0,0.0,...,0.2,623,2.0,9.0,10.0,115500.0,0.50,4.0,598.0,88.6
4,2224.0,Cu,36.5,138,ZnO,81.38,SBA-15,60,0,0.0,...,0.0,773,5.0,218.0,3.0,60000.0,0.04,4.0,453.0,60.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1167,1370.4,Cu,67.0,138,ZnO,81.38,Al2O3,101.96,0,0.0,...,0.0,623,3.0,96.0,4.0,48000.0,0.10,5.0,513.0,33.0
1168,382.4,Cu,67.0,138,ZnO,81.38,Al2O3,101.96,0,0.0,...,0.0,623,3.0,96.0,4.0,24000.0,0.10,4.0,473.0,33.0
1169,887.2,Cu,67.0,138,ZnO,81.38,Al2O3,101.96,0,0.0,...,0.0,623,3.0,96.0,4.0,42000.0,0.10,4.0,493.0,33.0
1170,1119.0,Cu,67.0,138,ZnO,81.38,Al2O3,101.96,0,0.0,...,0.0,623,3.0,96.0,4.0,42000.0,0.10,4.0,513.0,33.0


In [65]:
y = df.iloc[:, 0]
y = y/1000  #units aren't consistent

display(y)

0       2.6200
1       2.4000
2       2.2336
3       2.2300
4       2.2240
         ...  
1167    1.3704
1168    0.3824
1169    0.8872
1170    1.1190
1171    0.4400
Name: STY [mgMeOH h-1 gcat-1], Length: 1172, dtype: float64

In [66]:
import sklearn

X = (df[["Metal Loading [wt.%]", "Support Loading [wt.%]", "Promoter 1 loading [wt.%]", "Promoter 2 loading [wt.%]", "SBET [m2 g-1]", "H2/CO2 [-]", "GHSV [cm3 h-1 gcat-1]", "Catalyst amount [g]", "Pressure [Mpa]", "Temperature [K]", "Calcination Temperature [K]", "Calcination duration [h]"]]).copy()


display(X)

# rename the columns
column_names = ["M", "S", "PR1", "PR2", "S$_{BET}$", "H$_2$/CO$_2$", "GHSV", "MC", "P", "T", "CT", "CD"]
X.columns = column_names

X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size=0.15, random_state=10)

,Metal Loading [wt.%],Support Loading [wt.%],Promoter 1 loading [wt.%],Promoter 2 loading [wt.%],SBET [m2 g-1],H2/CO2 [-],GHSV [cm3 h-1 gcat-1],Catalyst amount [g],Pressure [Mpa],Temperature [K],Calcination Temperature [K],Calcination duration [h]
0,4.4,88.6,6.8,0.2,9.0,10.0,115500.0,0.50,6.0,598.0,623,2.0
1,4.4,88.6,6.8,0.2,9.0,8.0,94500.0,0.50,6.0,598.0,623,2.0
2,36.5,60.8,2.7,0.0,218.0,3.0,120000.0,0.02,4.0,453.0,573,4.0
3,4.4,88.6,6.8,0.2,9.0,10.0,115500.0,0.50,4.0,598.0,623,2.0
4,36.5,60.8,2.7,0.0,218.0,3.0,60000.0,0.04,4.0,453.0,773,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1167,67.0,33.0,0.0,0.0,96.0,4.0,48000.0,0.10,5.0,513.0,623,3.0
1168,67.0,33.0,0.0,0.0,96.0,4.0,24000.0,0.10,4.0,473.0,623,3.0
1169,67.0,33.0,0.0,0.0,96.0,4.0,42000.0,0.10,4.0,493.0,623,3.0
1170,67.0,33.0,0.0,0.0,96.0,4.0,42000.0,0.10,4.0,513.0,623,3.0


In [67]:
from sklearn.metrics import mean_squared_error, r2_score


my_xgb = Custom_xgb(n_estimators=30, max_depth=5, learning_rate=0.2, lambda_reg=1)


print("Training Custom XGBoost...")
my_xgb.fit(X_train, y_train) 
print("Done!")

preds = my_xgb.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, preds))
print(f"Custom XGB RMSE: {rmse}")

Training Custom XGBoost...
Done!
Custom XGB RMSE: 0.12340198085458358
